# struct 操作二进制数据

二进制对计算机来说即最直接的表示语言，我们所用的汉字英文或其他编码都有对应的二进制数据表示。

二进制数据通过一定的逻辑操作，实现了我们计算机的程序功能。

计算机上的文件，可以直接以二进制形式读取或编写。

对于不同格式的文件，都有其固定的二进制格式。


## 格式化字符串

格式字符串是在打包和解包数据时用于指定预期布局的机制。它们由格式字符构成，它指定了打包/解压缩数据的类型。另外，还有用于控制字节顺序，大小和对齐的特殊字符。

### 字节顺序，大小和对齐

默认情况下，C类型以机器的本机格式和字节顺序表示，并在必要时通过跳过填充字节（根据C编译器使用的规则）进行适当对齐。

或者，根据下表，格式字符串的第一个字符可用于指示打包数据的字节顺序，大小和对齐方式：

| 字符 | 字节顺序             | 大小 | 对准 |
| :--- | :------------------- | :--- | :--- |
| @    | 本地                 | 本地 | 本地 |
| =    | 本地                 | 标准 | 没有 |
| <    | 小--尾数             | 标准 | 没有 |
| >   | 大端                 | 标准 | 没有 |
| !    | 网络（= big-endian） | 标准 | 没有 |

如果第一个字符不是其中之一，则假定为“@”。

本地字节顺序是大端或小端，取决于主机系统。

使用`sys.byteorder`来检查你的系统的字节序。

原生大小和对齐是使用C编译器的`sizeof`表达式确定的。这总是与本地字节顺序相结合。

标准尺寸仅取决于格式字符; 请参阅格式字符部分中的表格。

注意'@'和'='之间的差别：都使用本地字节顺序，但后者的大小和对齐是标准化的。

表单`'!'`适用于那些声称自己不记得网络字节顺序是大端还是小端的穷人。

没有办法指示非本地字节顺序（强制字节交换）; 使用'<'或'>'的适当选择。

注意：

1. 填充只会自动添加在连续的结构成员之间。在编码结构的开头或结尾不添加填充。

2. 使用非原生大小和对齐时，例如'<'，'>'，'='和'！'不会添加填充。

3. 要将结构的末尾与特定类型的对齐要求对齐，请使用重复计数为零的该类型的代码结束格式。参见示例。

### 格式化字符

格式字符具有以下含义; 考虑到它们的类型，C和Python值之间的转换应该是显而易见的。 “标准大小”栏是指使用标准大小时打包值的大小（以字节为单位）; 也就是说，当格式字符串以'<'，'>'，'！'之一开始时 或'='。 使用原生尺寸时，打包值的大小取决于平台。

| 格式 | C 类型             | Python 类型        | 标准大小 | 注意     |
| :--- | :----------------- | :----------------- | :------- | :------- |
| x    | pad byte           | no value           |          |          |
| c    | char               | string of length 1 | 1        |          |
| b    | signed char        | integer            | 1        | (3)      |
| B    | unsigned char      | integer            | 1        | (3)      |
| ?    | _Bool              | bool               | 1        | (1)      |
| h    | short              | integer            | 2        | (3)      |
| H    | unsigned short     | integer            | 2        | (3)      |
| i    | int                | integer            | 4        | (3)      |
| I    | unsigned int       | integer            | 4        | (3)      |
| l    | long               | integer            | 4        | (3)      |
| L    | unsigned long      | integer            | 4        | (3)      |
| q    | long long          | integer            | 8        | (2), (3) |
| Q    | unsigned long long | integer            | 8        | (2), (3) |
| f    | float              | float              | 4        | (4)      |
| d    | double             | float              | 8        | (4)      |
| s    | char[]             | string             |          |          |
| p    | char[]             | string             |          |          |
| P    | void *             | integer            |          | (5), (3) |

注意：

1. '？' 转换代码对应于由C99定义的_Bool类型。 如果此类型不可用，则使用char模拟它。 在标准模式下，它总是由一个字节表示。

2.6版本中的新功能。

2. 将`'q'`和`'Q'`只有在平台C编译器支持C转换代码在本地模式中可用`long long`，或者在Windows上，`__int64`。它们始终可用于标准模式。

2.2版本中的新功能。

3. 当尝试使用任何整数转换代码打包非整数时，如果非整数具有__index __（）方法，则在打包之前调用该方法将参数转换为整数。 如果不存在__index __（）方法，或者对__index __（）的调用引发TypeError，则尝试__int __（）方法。 但是，__int __（）的使用已被弃用，并且会引发DeprecationWarning。

在版本2.7中更改：对非整数使用__index __（）方法是2.7中的新功能。2.7版中的更改：在版本2.7之前，并非所有整数转换代码都会使用__int __（）方法进行转换，并且引发了DeprecationWarning 仅用于float参数。

4. 对于`'f'`和`'d'`转换码，填充表示使用IEEE 754 binary32（对于`'f'`）或binary64（对于`'d'`）格式，而不管由所述平台中使用的浮点格式的。

5. 该`'P'`格式字符仅适用于本地字节顺序（选择为默认或与`'@'`字节顺序字符）。字节顺序字符`'='`选择使用基于主机系统的小端或大端排序。结构模块不会将其解释为本地排序，因此该`'P'`格式不可用。

格式字符之前可以有一个整数重复计数。 例如，格式字符串'4h'的含义与'hhhh'完全相同。

格式之间的空格字符被忽略; 计数及其格式不能包含空格。

对于's'格式字符，计数被解释为字符串的大小，而不是像其他格式字符一样重复计数; 例如，'10s'表示单个10字节的字符串，而'10c'表示10个字符。 如果没有给出计数，则默认为1.对于打包，字符串将被截断或填充为空字节，以使其合适。 对于解包，结果字符串总是具有指定的字节数。 作为一个特殊情况，'0'表示单个空字符串（而'0c'表示0个字符）。

'p'格式字符编码一个“Pascal字符串”，意思是一个简短的可变长度字符串，存储在固定数量的字节中，由count给出。 存储的第一个字节是字符串的长度，或255，以较小者为准。 字符串的字节在后面。 如果传递给pack（）的字符串太长（比计数减1还长），则仅存储字符串的前导count-1字节。 如果字符串比count-1短，则会填充空字节，以便使用全部的精确计数字节。 请注意，对于unpack（），'p'格式字符会消耗计数字节，但返回的字符串不能包含超过255个字符。

对于`'P'`格式字符，返回值是一个Python整数或长整数，具体取决于指针转换为整型时所需的大小。一个*NULL*指针将总是作为Python整数返回`0`。当打包指针大小的值时，可以使用Python整数或长整数对象。例如，Alpha和Merced处理器使用64位指针值，这意味着Python长整数将用于保存指针; 其他平台使用32位指针并将使用Python整数。

为了 '？' 格式字符，返回值是True或False。 打包时，使用参数对象的真值。 原生或标准bool表示中的0或1将被打包，并且在解包时任何非零值都将为True。

## 按格式打包解包

格式fmt，即类似于
```cpp
struct format{
  short s1,
  short s2,
  long l;
}
```

1. `pack(fmt,**kwargs)` 打包成字符到二进制字符串
2. `unpack(fmt,string)` 解包二进制字符串，返回元组
3. `calcsize(fmt)` 计算格式的位数

In [6]:
from struct import *
# 通过short,short,long>2+2+4 打包
print(pack('hhl', 1, 2, 3))
# 通过short,short,long>2+2+4 解包
print(unpack('hhl', b'\x00\x01\x00\x02\x00\x00\x00\x03'))
# 格式位数
print(calcsize('hhl'))

b'\x01\x00\x02\x00\x03\x00\x00\x00'
(256, 512, 50331648)
8


# 符号应用

符号，可以应用于填充位数的方式，默认为**@**

大端模式，是指数据的高字节保存在内存的低地址中，而数据的低字节保存在内存的高地址中，这样的存储模式有点儿类似于把数据当作字符串顺序处理：地址由小向大增加，而数据从高位往低位放；这和我们的阅读习惯一致。

小端模式，是指数据的高字节保存在内存的高地址中，而数据的低字节保存在内存的低地址中，这种存储模式将地址的高低和数据位权有效地结合起来，高地址部分权值高，低地址部分权值低。

In [15]:
print("无符号：",pack('hi',1,2))
print("@符号：",pack('@hi',1,2))
print("无符号即默认为@符号")
print(">符号：",pack('>hi',1,2))
print(">符号采用big-endian大端排列方式，尾部不会补全到hi的",calcsize('hi'),"实际只有",calcsize('>hi'))
print("<符号：",pack('<hi',1,2))
print("<符号采用little-endian小端排列方式，尾部不会补全到hi的",calcsize('hi'),"实际只有",calcsize('<hi'))
print("!符号：",pack('!hi',1,2))

无符号： b'\x01\x00\x00\x00\x02\x00\x00\x00'
@符号： b'\x01\x00\x00\x00\x02\x00\x00\x00'
无符号即默认为@符号
>符号： b'\x00\x01\x00\x00\x00\x02'
>符号采用big-endian大端排列方式，尾部不会补全到hi的 8 实际只有 6
<符号： b'\x01\x00\x02\x00\x00\x00'
<符号采用little-endian小端排列方式，尾部不会补全到hi的 8 实际只有 6
!符号： b'\x00\x01\x00\x00\x00\x02'


In [19]:
record = b'raymond   \x32\x12\x08\x01\x08'
name, serialnum, school, gradelevel = unpack('<10sHHb', record)

from collections import namedtuple
Student = namedtuple('Student', 'name serialnum school gradelevel')
# Student._make(unpack('<10sHHb', record))
# 或者
map(Student._make,unpack('<10sHHb', record))
Student(name='raymond   ', serialnum=4658, school=264, gradelevel=8)

Student(name='raymond   ', serialnum=4658, school=264, gradelevel=8)